In [2]:
# Load the dataset
import pandas as pd
train_df=pd.read_csv("DATASET/kindleReviewDataset.csv",index_col=0)
train_df=train_df.drop(columns=[i for i in train_df.columns if i not in ["reviewText","rating"]])

In [3]:
##Sum functions
print(train_df.isnull().sum(),"\n") #to find no of missing values
print(train_df['rating'].unique(),"\n") #types of unique ratings
print(train_df['rating'].value_counts()) #count per unique rating

rating        0
reviewText    0
dtype: int64 

[3 5 4 2 1] 

rating
5    3000
4    3000
3    2000
2    2000
1    2000
Name: count, dtype: int64


In [4]:
## postive review is 1 and negative review is 0
# train_df["rating"]=train_df["rating"].apply(lambda x: 0 if x<3 else 1)
train_df['rating'].value_counts()

rating
5    3000
4    3000
3    2000
2    2000
1    2000
Name: count, dtype: int64

In [5]:
## 1. Lower All the cases
train_df['reviewText']=train_df['reviewText'].str.lower()

In [6]:
import re
import nltk
from nltk.corpus import stopwords
# nltk.download('stopwords')
## Removing special characters (idk)
train_df['reviewText']=train_df['reviewText'].apply(lambda x:re.sub('[^a-z A-z 0-9-]+', '',x))
## Remove the stopswords
train_df['reviewText']=train_df['reviewText'].apply(lambda x: " ".join([y for y in x.split() if y not in stopwords.words('english')]))
## Remove url (idk)
train_df['reviewText']=train_df['reviewText'].apply(lambda x: re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '' , str(x)))
## Remove html tags (idk)
# from bs4 import BeautifulSoup
# train_df['reviewText']=train_df['reviewText'].apply(lambda x: BeautifulSoup(x, 'lxml').get_text())
## Remove any additional spaces
train_df['reviewText']=train_df['reviewText'].apply(lambda x: " ".join(x.split()))

In [7]:
train_df.head()

,rating,reviewText
0,3,jace rankin may short hes nothing mess man hau...
1,5,great short read didnt want put read one sitti...
2,3,ill start saying first four books wasnt expect...
3,3,aggie angela lansbury carries pocketbooks inst...
4,4,expect type book library pleased find price right


In [8]:
## Lemmatizer (root words basically)
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])
train_df['reviewText']=train_df['reviewText'].apply(lemmatize_words)

In [9]:
train_df.head()

,rating,reviewText
0,3,jace rankin may short he nothing mess man haul...
1,5,great short read didnt want put read one sitti...
2,3,ill start saying first four book wasnt expecti...
3,3,aggie angela lansbury carry pocketbook instead...
4,4,expect type book library pleased find price right


In [10]:
## Train Test Split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(train_df['reviewText'],train_df['rating'],
                                              test_size=0.20,random_state=69)

In [11]:
# Feature Extraction Techinque
# By Bag Of Words : basically concentrated OHE
from sklearn.feature_extraction.text import CountVectorizer
bow=CountVectorizer()
X_train_bow=bow.fit_transform(X_train).toarray()
X_test_bow=bow.transform(X_test).toarray()

In [12]:
X_train_bow[0]

array([0, 0, 0, ..., 0, 0, 0])

In [13]:
# Feature Extraction Techinque
# tfidf: BOW me 1 ki jagah a calculated relative value
#       (captures importance of a word)
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer()
X_train_tfidf=tfidf.fit_transform(X_train).toarray()
X_test_tfidf=tfidf.transform(X_test).toarray()

In [14]:
# Naive-Bayes : Classification Technique
#works well with small datasets and bow/tfidf
#it works on X_train only ofc! not dependent on how 
#    many classses y_train have, they all support
#    multiclass classification.

#BernoulliNB: for if only presence of feature matters
#   like suppose x_train[0]:[0,2,1,0]=>[0,1,1,0]
#   useful in say spam classificaiton(we need whether spam word present or not)
#  -movie review me positive review me suppose fantastic 5 baar aa raha hai
#       aur boring ek baar, par dono ki keemat same=>unfair for data like these
  
# MultiNomialNB: for when frequency matters too
#   like x_train[0]:[0,2,1,0]=>[0,2,1,0]
#   works best with textual data(like this example)
#   -fantastic ki keemat zyada hogi,
  
# gusssian dist!=>countinous
# GaussianNB: for when data is in guassian dist,
#   i.e., it follows bell curve
#  -suppose if most movie reviews are 2-3 out of 5,
#   then gnb will work better than mnb

from sklearn.naive_bayes import BernoulliNB
bnb_model_bow=BernoulliNB().fit(X_train_bow,y_train)
bnb_model_tfidf=BernoulliNB().fit(X_train_tfidf,y_train)
from sklearn.naive_bayes import MultinomialNB
mnb_model_bow=MultinomialNB().fit(X_train_bow,y_train)
mnb_model_tfidf=MultinomialNB().fit(X_train_tfidf,y_train)
from sklearn.naive_bayes import GaussianNB
gnb_model_bow=GaussianNB().fit(X_train_bow,y_train)
gnb_model_tfidf=GaussianNB().fit(X_train_tfidf,y_train)

In [15]:
y_pred_bow_bnb  =bnb_model_bow.predict(X_test_bow)
y_pred_bow_mnb  =mnb_model_bow.predict(X_test_bow)
y_pred_bow_gnb  =gnb_model_bow.predict(X_test_bow)

In [16]:
y_pred_tfidf_bnb= bnb_model_bow.predict(X_test_tfidf)
y_pred_tfidf_mnb= mnb_model_bow.predict(X_test_tfidf)
y_pred_tfidf_gnb= gnb_model_bow.predict(X_test_tfidf)

In [18]:
from sklearn.metrics import accuracy_score,confusion_matrix
print("BOW accuracy:bnb ",accuracy_score(y_test,y_pred_bow_bnb))
print("BOW accuracy:mnb ",accuracy_score(y_test,y_pred_bow_mnb))
print("BOW accuracy:gnb ",accuracy_score(y_test,y_pred_bow_gnb))

BOW accuracy:bnb  0.39458333333333334
BOW accuracy:mnb  0.48541666666666666
BOW accuracy:gnb  0.31


In [19]:
print("TFIDF accuracy:bnb ",accuracy_score(y_test,y_pred_tfidf_bnb))
print("TFIDF accuracy:mnb ",accuracy_score(y_test,y_pred_tfidf_mnb))
print("TFIDF accuracy:gnb ",accuracy_score(y_test,y_pred_tfidf_gnb))

TFIDF accuracy:bnb  0.39458333333333334
TFIDF accuracy:mnb  0.4633333333333333
TFIDF accuracy:gnb  0.3120833333333333


In [21]:
from sklearn.metrics import confusion_matrix,accuracy_score

## How to read confusion matrix
#diagonal elements: Correct predictions.
#off-diagonal elements: Misclassifications.
# suppose for row 1(class 1):
# 204 times, class 1 was correctly predicted as 1.
# 72 times, class 1 was misclassified as class 2.
# 50 times, class 2 was misclassified as class 5.
# 191 times, class 4 was correctly predicted as 4.
# 92 times, class 5 was misclassified as class 4.
print(f"{confusion_matrix(y_test,y_pred_tfidf_bnb)}")
print(f"{confusion_matrix(y_test,y_pred_tfidf_mnb)}")
print(f"{confusion_matrix(y_test,y_pred_tfidf_gnb)}")
#the confusion matrix is 5*5 since len(y_test.unique())=5

[[144  41  14  36 176]
 [ 62  76  41  58 183]
 [ 26  35  45 120 196]
 [  3   5  22 231 306]
 [  2   3   9 115 451]]
[[171 110  20  87  23]
 [ 78 154  56 103  29]
 [ 29  55  69 219  50]
 [  6  12  21 374 154]
 [  6   6   7 217 344]]
[[204  72  44  62  29]
 [166  88  44  72  50]
 [146  59  82  78  57]
 [147  70  68 191  91]
 [144  87  73  92 184]]
